In [1]:
import dataIO_funcs
# from datetime import datetime
import datetime
import numpy as np
import dateutil.parser
import os
import subprocess
from snowexsql.db import get_db
from snowexsql.data import LayerData,PointData# Import the function to get connect to the db
from snowexsql.conversions import query_to_geopandas # Import a useful function to format that data into a dataframe 
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, Polygon

In [20]:
# specify paths
bucket_name = 'eis-dh-hydro/SNOWEX-HACKWEEK'
LIS_path = f's3://{bucket_name}/2022/ZARR/SURFACEMODEL/LIS_HIST_default_chunks.d01.zarr/'
# LIS_path = f's3://{bucket_name}/2022/ZARR/SURFACEMODEL/LIS_HIST_rechunkedV4.d01.zarr'
SWESARR_url = 'https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease/'

# SWESARR data website
source_repo = 'https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease/'

# specify filters
time_sel='2020-02-08'
lat_range = [38.8, 38.9]
lon_range = [-108.5, -107.5]
model_variable = ['SM_SWE_inst','SM_SnowDepth_inst']
dx = 0.0011
dy = 0.0009
time_buffer_dy = 2

db_name = 'snow:hackweek@db.snowexdata.org/snowex' # This is what you will use for all of hackweek to access the db
type_name = 'point'
var_name = 'swe'


# FOR PULLING SWESARR
#### TAKES A LOOK AT ALL OF THE DATA AVAILABLE, THEN ATTAEMPTS TO PULL THE DATA THAT IS 'TIME_BUFFER_DY' FROM THE SPECIFIED DAY 'TIME_SEL'

In [3]:
# pull the dates that SWESARR occurred
dataIO_funcs.access_SWESAR
SWESARR_names,SWESARR_dates = dataIO_funcs.access_SWESARR.get_url_paths(SWESARR_url)
print(SWESARR_names)

AttributeError: module 'dataIO_funcs' has no attribute 'access_SWESAR'

In [13]:
def date_between_prime(start_date, end_date, folder_date,folder_names):
    result = [folder_names[i] for i in range(len(folder_date)) if (folder_date[i] >= start_date and folder_date[i] <= end_date)]
    return result
time_sel_newForm = dateutil.parser.parse(time_sel)
start_date = time_sel_newForm-datetime.timedelta(days=time_buffer_dy)
end_date = time_sel_newForm+datetime.timedelta(days=time_buffer_dy)

final_files = date_between_prime(start_date, end_date, SWESARR_dates, SWESARR_names)
for flight_line in final_files:
    print(source_repo + flight_line)
    # command = wget -r -np -nH --reject "indexd.html*" -e robots=off -r --no-parent -A "*tif" source_repo + flight_line
    subprocess.Popen(['wget -r -np -nH --reject indexd.html* -e robots=off -r --no-parent -A *tif',source_repo+flight_line], shell=True)
    # ! wget -r -np -nH --reject "indexd.html*" -e robots=off -r --no-parent -A "*tif" source_repo + flight_line
    
data_files = []
os.chdir(os.getcwd() + flight_line)
for file in glob.glob("*.tif"):
    data_files.append(file)
    
data_files = [data for data in data_files if data[-8:] != '_dem.tif']

https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease//GRMCT2_13802_20006_012_200210_225_XX_01/
https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease//GRMCT2_31601_20006_011_200210_225_XX_01/
https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease//GRMNT1_09302_20006_009_200210_225_XX_01/
https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease//GRMNT1_09401_20006_005_200210_225_XX_01/
https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease//GRMST1_27302_20006_007_200210_225_XX_01/
https://glihtdata.gsfc.nasa.gov/files/radar/SWESARR/prerelease//GRMST1_27501_20006_003_200210_225_XX_01/


wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/model-eval/contributors/jupflug/GRMST1_27501_20006_003_200210_225_XX_01/'

# LIS MODEL READ-IN
#### READS IN THE DATA BASED ON THE LAT/LON/DATE CONSTRAINTS, PROVIDED THE MODEL VARIABLES OF FOCUS
#### WE ALSO GIVE IT THE APPROXIMATE MODEL DX AND DY WE WANT TO MOVE TO WHEN CONVERTING TO A RECTANGULAR GRID

In [3]:
ds_LIS = dataIO_funcs.access_LIS(lon_range,lat_range,time_sel,
                             model_variable,LIS_path,dx,dy,'bilinear')

In [3]:
ds_LIS

NameError: name 'ds_LIS' is not defined

# POINT AND LAYER DATA READ-IN

In [3]:
df = dataIO_funcs.access_snowEx.access_pointData(db_name,time_sel,time_buffer_dy,var_name,lat_range,lon_range)

(datetime.date(2020, 2, 8),)
(datetime.date(2020, 2, 9),)
(datetime.date(2020, 2, 6),)
(datetime.date(2020, 2, 10),)
(datetime.date(2020, 2, 7),)
[[-107.7, 38.8], [-107.5, 38.8], [-107.5, 38.9], [-107.7, 38.9]]
